In [449]:
from datetime import datetime, timedelta
import calendar
import cloudscraper
from bs4 import BeautifulSoup
import pandas as pd
import requests
from message_data import message_data_array

In [450]:
#import datetime
current_year = str(datetime.now().year)
current_month = str(datetime.now().strftime('%B').capitalize())
#print(current_year)
#print(current_month)

# Get the current day
today = datetime.now()
first = today.replace(day=1)

# Format the date
today = str(int(today.strftime('%d')))
first = str(int(first.strftime('%d')))
end = str(calendar.monthrange(int(current_year), int(datetime.strptime(current_month, '%B').month))[1])

#print('first day of a month:', first)
#print('Last day of the month is:', end)
#print('Today is:', today)

In [451]:
#import cloudscraper
#from bs4 import BeautifulSoup
#import pandas as pd
with cloudscraper.create_scraper(delay=10, browser={'custom': 'ScraperBot/1.0'}) as scraper:
    r = scraper.get('https://www.forexfactory.com/calendar')

    soup = BeautifulSoup(r.text, 'html.parser')
    #store the security value in a variable incase it changes
    #from the html code
    security_value = soup.find('input', {'name': 'flex[Calendar_mainCal][modelData]'})['value'] 

    #print(security_value)

    #from developer tools > network > flex.php (main network) > payload > view_source
    data = f'securitytoken=guest&do=saveoptions&setdefault=no&ignoreinput=no&flex%5BCalendar_mainCalCopy1%5D%5BidSuffix%5D=&flex%5BCalendar_mainCalCopy1%5D%5B_flexForm_%5D=flexForm&flex%5BCalendar_mainCalCopy1%5D%5BmodelData%5D={security_value}&flex%5BCalendar_mainCalCopy1%5D%5Bbegindate%5D={current_month}+{first}%2C+{current_year}&flex%5BCalendar_mainCalCopy1%5D%5Benddate%5D={current_month}+{end}%2C+{current_year}&flex%5BCalendar_mainCalCopy1%5D%5Bcalendardefault%5D=today&flex%5BCalendar_mainCalCopy1%5D%5Bimpacts%5D%5Bhigh%5D=high&flex%5BCalendar_mainCalCopy1%5D%5B_cbarray_%5D=1&flex%5BCalendar_mainCalCopy1%5D%5Beventtypes%5D%5Bgrowth%5D=growth&flex%5BCalendar_mainCalCopy1%5D%5Beventtypes%5D%5Binflation%5D=inflation&flex%5BCalendar_mainCalCopy1%5D%5Beventtypes%5D%5Bemployment%5D=employment&flex%5BCalendar_mainCalCopy1%5D%5Beventtypes%5D%5Bcentralbank%5D=centralbank&flex%5BCalendar_mainCalCopy1%5D%5Beventtypes%5D%5Bbonds%5D=bonds&flex%5BCalendar_mainCalCopy1%5D%5Beventtypes%5D%5Bhousing%5D=housing&flex%5BCalendar_mainCalCopy1%5D%5Beventtypes%5D%5Bsentiment%5D=sentiment&flex%5BCalendar_mainCalCopy1%5D%5Beventtypes%5D%5Bpmi%5D=pmi&flex%5BCalendar_mainCalCopy1%5D%5Beventtypes%5D%5Bspeeches%5D=speeches&flex%5BCalendar_mainCalCopy1%5D%5Beventtypes%5D%5Bmisc%5D=misc&flex%5BCalendar_mainCalCopy1%5D%5B_cbarray_%5D=1&flex%5BCalendar_mainCalCopy1%5D%5Bcurrencies%5D%5Beur%5D=eur&flex%5BCalendar_mainCalCopy1%5D%5Bcurrencies%5D%5Bgbp%5D=gbp&flex%5BCalendar_mainCalCopy1%5D%5Bcurrencies%5D%5Busd%5D=usd&flex%5BCalendar_mainCalCopy1%5D%5B_cbarray_%5D=1&false'
    
    #copy the request headers
    #exclude use-agent since cloud scrapper already covers it
    headers = {
        'authority': 'www.forexfactory.com',
        'method': 'POST',
        'path': '/flex.php?',
        'scheme': 'https',
        'accept': '/',
        'accept-encoding': 'gzip, deflate, br',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
        'content-length': '1974',
        'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
        'origin': 'https://www.forexfactory.com',
        'referer': 'https://www.forexfactory.com/calendar',
        'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"macOS"',
        'sec-fetch-dest': 'empty',
        'sec-fetch-mode': 'cors',
        'sec-fetch-site': 'same-origin',
        'x-requested-with': 'XMLHttpRequest'
    }
    r = scraper.post('https://www.forexfactory.com/flex.php?', data=data, headers=headers) #use request URL

    soup = BeautifulSoup(r.text,'lxml')

    calendar_table = soup.find('table', class_="calendar__table")

    #print(calendar_table)
    column_headers = ['Date', 'Time', 'void', 'Currency', 'Impact', 'Event', 'Detail_void', 'Actual', 'Forecast', 'Previous', 'Graph_void']
    df = pd.DataFrame(columns = column_headers)
    for row in calendar_table.select('tr[class*="calendar__row calendar_row"]'):
        row_data = [td.get_text(strip=True) for td in row.find_all('td')]
        df.loc[len(df)] = row_data
df

,Date,Time,void,Currency,Impact,Event,Detail_void,Actual,Forecast,Previous,Graph_void
0,WedFeb 1,8:15am,,USD,,ADP Non-Farm Employment Change,,106K,176K,253K,
1,,10:00am,,USD,,ISM Manufacturing PMI,,47.4,48.0,48.4,
2,,,,USD,,JOLTS Job Openings,,11.01M,10.28M,10.44M,
3,,2:00pm,,USD,,FOMC Statement,,,,,
4,,,,USD,,Federal Funds Rate,,4.75%,4.75%,4.50%,
5,,2:30pm,,USD,,FOMC Press Conference,,,,,
6,ThuFeb 2,7:00am,,GBP,,BOE Monetary Policy Report,,,,,
7,,,,GBP,,MPC Official Bank Rate Votes,,7-0-2,7-0-2,7-0-2,
8,,,,GBP,,Monetary Policy Summary,,,,,
9,,,,GBP,,Official Bank Rate,,4.00%,4.00%,3.50%,


In [452]:
#drop the columns with no data
df.drop(columns=['void', 'Detail_void', 'Graph_void', 'Impact'], inplace=True)

In [453]:
#Deletion and imputation
df['Date'].replace(to_replace='', value=None, method='ffill', inplace=True) #fill empty dates with previous data
df['Time'].replace(to_replace='', value=None, method='ffill', inplace=True) #fill empty times with previous data
df.drop(labels=df[df['Event'] == ''].index, inplace=True) #drop rows with no events
df.insert(len(df.columns), 'Effect', None) #insert column 'Effect' at the end
df.insert(len(df.columns), 'why_it_matters', None) #insert column 'why_it_matters' at the end
import datetime
df['Date'] = df['Date'].apply(lambda x: datetime.datetime.strptime(x, '%a%b %d').strftime('%a, %b %d'))
#df

In [454]:
df2 = pd.read_csv('extra_data.csv')
#df2

In [455]:
merged_df = df.merge(df2, on='Event', how='left')

merged_df['Effect'] = merged_df['Effect_y'].fillna(merged_df['Effect_x'])
merged_df['why_it_matters'] = merged_df['why_it_matters_y'].fillna(merged_df['why_it_matters_x'])

merged_df = merged_df.drop(columns=['Effect_x','Effect_y','why_it_matters_x','why_it_matters_y'])

merged_df

,Date,Time,Currency,Event,Actual,Forecast,Previous,Effect,why_it_matters
0,"Thu, Feb 01",8:15am,USD,ADP Non-Farm Employment Change,106K,176K,253K,Actual greater than Forecast is good for the m...,Job creation is an important leading indicator...
1,"Thu, Feb 01",10:00am,USD,ISM Manufacturing PMI,47.4,48.0,48.4,Actual greater than Forecast is good for the m...,It's a leading indicator of economic health - ...
2,"Thu, Feb 01",10:00am,USD,JOLTS Job Openings,11.01M,10.28M,10.44M,Actual greater than Forecast is good for the m...,"It's released late, but can impact the market ..."
3,"Thu, Feb 01",2:00pm,USD,FOMC Statement,,,,More hawkish than expected is good for market,It's the primary tool the FOMC uses to communi...
4,"Thu, Feb 01",2:00pm,USD,Federal Funds Rate,4.75%,4.75%,4.50%,Actual greater than Forecast is good for the m...,Short term interest rates are the paramount fa...
5,"Thu, Feb 01",2:30pm,USD,FOMC Press Conference,,,,More hawkish than expected is good for market,It's among the primary methods the Fed uses to...
6,"Fri, Feb 02",7:00am,GBP,BOE Monetary Policy Report,,,,More hawkish than expected is good for market,It provides valuable insight into the bank's v...
7,"Fri, Feb 02",7:00am,GBP,MPC Official Bank Rate Votes,7-0-2,7-0-2,7-0-2,Actual greater than Forecast is good for the m...,The BOE's MPC meeting minutes contain the inte...
8,"Fri, Feb 02",7:00am,GBP,Monetary Policy Summary,,,,More hawkish than expected is good for market,It's among the primary tools the MPC uses to c...
9,"Fri, Feb 02",7:00am,GBP,Official Bank Rate,4.00%,4.00%,3.50%,Actual greater than Forecast is good for the m...,Short term interest rates are the paramount fa...


In [456]:
# Define the time zone offsets
#USA zone
gmt_minus_5 = datetime.timezone(datetime.timedelta(hours=-5))
#Spain zone
gmt_plus_1 = datetime.timezone(datetime.timedelta(hours=1))

# Define a function to convert the time
def convert_time(time_str, from_time_zone, to_time_zone):
    # Parse the time string into a datetime object using the GMT-5 timezone
    time = datetime.datetime.strptime(time_str, '%I:%M%p').replace(tzinfo=from_time_zone)
    # Convert the datetime object to the GMT+1 timezone
    time = time.astimezone(to_time_zone)
    # Format the time as a string and return it
    return time.strftime('%I:%M%p')

# Apply the function to the 'Time' column and make new EuropeTime column
merged_df['Europe_time'] = merged_df['Time'].apply(convert_time, args=(gmt_minus_5, gmt_plus_1))

#make new column with EuropeTime as military time
merged_df['Europe_time_military'] = merged_df['Europe_time'].apply(lambda x: datetime.datetime.strptime(x, '%I:%M%p').strftime('%H:%M'))
#print the data types


In [457]:

merged_df.drop(columns=['Europe_time'], inplace=True)
merged_df


,Date,Time,Currency,Event,Actual,Forecast,Previous,Effect,why_it_matters,Europe_time_military
0,"Thu, Feb 01",8:15am,USD,ADP Non-Farm Employment Change,106K,176K,253K,Actual greater than Forecast is good for the m...,Job creation is an important leading indicator...,14:15
1,"Thu, Feb 01",10:00am,USD,ISM Manufacturing PMI,47.4,48.0,48.4,Actual greater than Forecast is good for the m...,It's a leading indicator of economic health - ...,16:00
2,"Thu, Feb 01",10:00am,USD,JOLTS Job Openings,11.01M,10.28M,10.44M,Actual greater than Forecast is good for the m...,"It's released late, but can impact the market ...",16:00
3,"Thu, Feb 01",2:00pm,USD,FOMC Statement,,,,More hawkish than expected is good for market,It's the primary tool the FOMC uses to communi...,20:00
4,"Thu, Feb 01",2:00pm,USD,Federal Funds Rate,4.75%,4.75%,4.50%,Actual greater than Forecast is good for the m...,Short term interest rates are the paramount fa...,20:00
5,"Thu, Feb 01",2:30pm,USD,FOMC Press Conference,,,,More hawkish than expected is good for market,It's among the primary methods the Fed uses to...,20:30
6,"Fri, Feb 02",7:00am,GBP,BOE Monetary Policy Report,,,,More hawkish than expected is good for market,It provides valuable insight into the bank's v...,13:00
7,"Fri, Feb 02",7:00am,GBP,MPC Official Bank Rate Votes,7-0-2,7-0-2,7-0-2,Actual greater than Forecast is good for the m...,The BOE's MPC meeting minutes contain the inte...,13:00
8,"Fri, Feb 02",7:00am,GBP,Monetary Policy Summary,,,,More hawkish than expected is good for market,It's among the primary tools the MPC uses to c...,13:00
9,"Fri, Feb 02",7:00am,GBP,Official Bank Rate,4.00%,4.00%,3.50%,Actual greater than Forecast is good for the m...,Short term interest rates are the paramount fa...,13:00


In [471]:
from datetime import datetime
year = datetime.now().strftime('%Y')
merged_df['Datetime'] = merged_df['Date'] + ' ' + year + ', ' + merged_df['Time']
merged_df

,Date,Time,Currency,Event,Actual,Forecast,Previous,Effect,why_it_matters,Europe_time_military,Datetime
0,"Thu, Feb 01",8:15am,USD,ADP Non-Farm Employment Change,106K,176K,253K,Actual greater than Forecast is good for the m...,Job creation is an important leading indicator...,14:15,"Thu, Feb 01 2023, 8:15am"
1,"Thu, Feb 01",10:00am,USD,ISM Manufacturing PMI,47.4,48.0,48.4,Actual greater than Forecast is good for the m...,It's a leading indicator of economic health - ...,16:00,"Thu, Feb 01 2023, 10:00am"
2,"Thu, Feb 01",10:00am,USD,JOLTS Job Openings,11.01M,10.28M,10.44M,Actual greater than Forecast is good for the m...,"It's released late, but can impact the market ...",16:00,"Thu, Feb 01 2023, 10:00am"
3,"Thu, Feb 01",2:00pm,USD,FOMC Statement,,,,More hawkish than expected is good for market,It's the primary tool the FOMC uses to communi...,20:00,"Thu, Feb 01 2023, 2:00pm"
4,"Thu, Feb 01",2:00pm,USD,Federal Funds Rate,4.75%,4.75%,4.50%,Actual greater than Forecast is good for the m...,Short term interest rates are the paramount fa...,20:00,"Thu, Feb 01 2023, 2:00pm"
5,"Thu, Feb 01",2:30pm,USD,FOMC Press Conference,,,,More hawkish than expected is good for market,It's among the primary methods the Fed uses to...,20:30,"Thu, Feb 01 2023, 2:30pm"
6,"Fri, Feb 02",7:00am,GBP,BOE Monetary Policy Report,,,,More hawkish than expected is good for market,It provides valuable insight into the bank's v...,13:00,"Fri, Feb 02 2023, 7:00am"
7,"Fri, Feb 02",7:00am,GBP,MPC Official Bank Rate Votes,7-0-2,7-0-2,7-0-2,Actual greater than Forecast is good for the m...,The BOE's MPC meeting minutes contain the inte...,13:00,"Fri, Feb 02 2023, 7:00am"
8,"Fri, Feb 02",7:00am,GBP,Monetary Policy Summary,,,,More hawkish than expected is good for market,It's among the primary tools the MPC uses to c...,13:00,"Fri, Feb 02 2023, 7:00am"
9,"Fri, Feb 02",7:00am,GBP,Official Bank Rate,4.00%,4.00%,3.50%,Actual greater than Forecast is good for the m...,Short term interest rates are the paramount fa...,13:00,"Fri, Feb 02 2023, 7:00am"


In [480]:
merged_df['Datetime'] = pd.to_datetime(merged_df['Datetime'], format='%a, %b %d %Y, %I:%M%p')
merged_df

,Date,Time,Currency,Event,Actual,Forecast,Previous,Effect,why_it_matters,Europe_time_military,Datetime
0,"Thu, Feb 01",8:15am,USD,ADP Non-Farm Employment Change,106K,176K,253K,Actual greater than Forecast is good for the m...,Job creation is an important leading indicator...,14:15,2023-02-01 08:15:00
1,"Thu, Feb 01",10:00am,USD,ISM Manufacturing PMI,47.4,48.0,48.4,Actual greater than Forecast is good for the m...,It's a leading indicator of economic health - ...,16:00,2023-02-01 10:00:00
2,"Thu, Feb 01",10:00am,USD,JOLTS Job Openings,11.01M,10.28M,10.44M,Actual greater than Forecast is good for the m...,"It's released late, but can impact the market ...",16:00,2023-02-01 10:00:00
3,"Thu, Feb 01",2:00pm,USD,FOMC Statement,,,,More hawkish than expected is good for market,It's the primary tool the FOMC uses to communi...,20:00,2023-02-01 14:00:00
4,"Thu, Feb 01",2:00pm,USD,Federal Funds Rate,4.75%,4.75%,4.50%,Actual greater than Forecast is good for the m...,Short term interest rates are the paramount fa...,20:00,2023-02-01 14:00:00
5,"Thu, Feb 01",2:30pm,USD,FOMC Press Conference,,,,More hawkish than expected is good for market,It's among the primary methods the Fed uses to...,20:30,2023-02-01 14:30:00
6,"Fri, Feb 02",7:00am,GBP,BOE Monetary Policy Report,,,,More hawkish than expected is good for market,It provides valuable insight into the bank's v...,13:00,2023-02-02 07:00:00
7,"Fri, Feb 02",7:00am,GBP,MPC Official Bank Rate Votes,7-0-2,7-0-2,7-0-2,Actual greater than Forecast is good for the m...,The BOE's MPC meeting minutes contain the inte...,13:00,2023-02-02 07:00:00
8,"Fri, Feb 02",7:00am,GBP,Monetary Policy Summary,,,,More hawkish than expected is good for market,It's among the primary tools the MPC uses to c...,13:00,2023-02-02 07:00:00
9,"Fri, Feb 02",7:00am,GBP,Official Bank Rate,4.00%,4.00%,3.50%,Actual greater than Forecast is good for the m...,Short term interest rates are the paramount fa...,13:00,2023-02-02 07:00:00


In [491]:
import datetime
#date_actual = datetime.datetime.now().strftime('%a, %b %d %Y, %I:%M%p')
date_actual = (datetime.datetime.now() - datetime.timedelta(hours=6)).strftime('%a, %b %d %Y, %I:%M%p')
date_actual = pd.to_datetime(date_actual, format='%a, %b %d %Y, %I:%M%p')
Datetime_var = merged_df.iloc[0]['Datetime']
print(date_actual)



2023-02-21 17:42:00


In [492]:
# Create an empty list to store the rows that meet the condition
actual_data_rows = []

# Loop through the rows of the DataFrame
for index, row in merged_df.iterrows():
    
    # Check if the date in the row is greater than the current date AND
    # if the time in the row is greater than the current time
    if row['Datetime'] < date_actual or row['Datetime'] == date_actual:
        # If it is, append the row to the list of actual_data_rows
        actual_data_rows.append(row)

# Convert the list of actual_data_rows to a DataFrame
actual_data_rows = pd.DataFrame(actual_data_rows)

# Extract the time component of the Time column
#actual_data_rows['Europe_time_military'] = actual_data_rows['Europe_time_military'].dt.time

# Print the resulting DataFrame
actual_data_rows

,Date,Time,Currency,Event,Actual,Forecast,Previous,Effect,why_it_matters,Europe_time_military,Datetime
0,"Thu, Feb 01",8:15am,USD,ADP Non-Farm Employment Change,106K,176K,253K,Actual greater than Forecast is good for the m...,Job creation is an important leading indicator...,14:15,2023-02-01 08:15:00
1,"Thu, Feb 01",10:00am,USD,ISM Manufacturing PMI,47.4,48.0,48.4,Actual greater than Forecast is good for the m...,It's a leading indicator of economic health - ...,16:00,2023-02-01 10:00:00
2,"Thu, Feb 01",10:00am,USD,JOLTS Job Openings,11.01M,10.28M,10.44M,Actual greater than Forecast is good for the m...,"It's released late, but can impact the market ...",16:00,2023-02-01 10:00:00
3,"Thu, Feb 01",2:00pm,USD,FOMC Statement,,,,More hawkish than expected is good for market,It's the primary tool the FOMC uses to communi...,20:00,2023-02-01 14:00:00
4,"Thu, Feb 01",2:00pm,USD,Federal Funds Rate,4.75%,4.75%,4.50%,Actual greater than Forecast is good for the m...,Short term interest rates are the paramount fa...,20:00,2023-02-01 14:00:00
5,"Thu, Feb 01",2:30pm,USD,FOMC Press Conference,,,,More hawkish than expected is good for market,It's among the primary methods the Fed uses to...,20:30,2023-02-01 14:30:00
6,"Fri, Feb 02",7:00am,GBP,BOE Monetary Policy Report,,,,More hawkish than expected is good for market,It provides valuable insight into the bank's v...,13:00,2023-02-02 07:00:00
7,"Fri, Feb 02",7:00am,GBP,MPC Official Bank Rate Votes,7-0-2,7-0-2,7-0-2,Actual greater than Forecast is good for the m...,The BOE's MPC meeting minutes contain the inte...,13:00,2023-02-02 07:00:00
8,"Fri, Feb 02",7:00am,GBP,Monetary Policy Summary,,,,More hawkish than expected is good for market,It's among the primary tools the MPC uses to c...,13:00,2023-02-02 07:00:00
9,"Fri, Feb 02",7:00am,GBP,Official Bank Rate,4.00%,4.00%,3.50%,Actual greater than Forecast is good for the m...,Short term interest rates are the paramount fa...,13:00,2023-02-02 07:00:00


In [495]:
#import requests
#from message_data import message_data_array
# Identify the rows that have data in the 'Actual' column
actual_data_rows = merged_df[merged_df['Actual'] != '']
# Iterate over the rows with data in the 'Actual' column
for index, row in actual_data_rows.iterrows():

	# Create a string with all the data from the row
	data_string = 'Date : {}, Time : {}, Currency : {}, Event : {}, Actual : {}, Forecast : {}, Previous : {}, Effect : {},  why_it_matters : {}'.format(row['Date'], row['Europe_time_military'], row['Currency'], row['Event'], row['Actual'], row['Forecast'], row['Previous'], row['Effect'], row['why_it_matters'])
	#print(data_string) #store each one in an array and 
	if data_string not in message_data_array:
		message_data_array.append(data_string)
		#write array back to the file
		f = open("message_data.py", "w")
		f.write("message_data_array = " + str(message_data_array))
		f.close()   #this will overwrite the existing file
		# Send the data from the 'Actual' column to the desired channel in Slack
		requests.post(url='your webhook',  json={
	"blocks": [
		{
			"type": "header",
			"text": {
				"type": "plain_text",
				"text": "Macro News!"
			}
		},
		{
			"type": "section",
			"text": {
				"type": "plain_text",
				"text": f"{row['Currency']}: {row['Event']}"
			}
		},
		{
			"type": "section",
			"fields": [
				{
					"type": "mrkdwn",
					"text": f"*Result:*\n{row['Actual']}"
				},
				{
					"type": "mrkdwn",
					"text": f"*Forecast:*\n{row['Forecast']}"
				},
				{
					"type": "mrkdwn",
					"text": f"*Previous:*\n{row['Previous']}"
				},
				{
					"type": "mrkdwn",
					"text": f"*Effect:*\n{row['Effect']}"
				},
				{
					"type": "mrkdwn",
					"text": f"*Why it matters:*\n{row['why_it_matters']}"
				}
			]
		},
		{
			"type": "context",
			"elements": [
				{
					"type": "plain_text",
					"text": f"Date: {row['Date']}"
				},
				{
					"type": "plain_text",
					"text": f"Time: {row['Europe_time_military']}"
				}
			]
		},
		{
			"type": "divider"
		},
		{
			"type": "context",
			"elements": [
				{
					"type": "image",
					"image_url": "https://www.linkpicture.com/q/1645628917769.jpeg",
					"alt_text": "EDEN_logo"
				},
				{
					"type": "mrkdwn",
					"text": "Economic Webscraper by *Gabriel de Olaguibel*"
				}
			]
		}
	]
})

In [52]:
'''
#import message_data file
from message_data import message_data_array

def deleteLastElement():
    #delete last element from the array
    message_data_array.pop()

    #write array back to the file
    f = open("message_data.py", "w")
    f.write("message_data_array = " + str(message_data_array))
    f.close()   #this will overwrite the existing file
deleteLastElement()
'''


'\n#import message_data file\nfrom message_data import message_data_array\n\ndef deleteLastElement():\n    #delete last element from the array\n    message_data_array.pop()\n\n    #write array back to the file\n    f = open("message_data.py", "w")\n    f.write("message_data_array = " + str(message_data_array))\n    f.close()   #this will overwrite the existing file\ndeleteLastElement()\n'